In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.0.2
scikit-learn-intelex==2021.20210714.120553


In [58]:
import pickle
import pandas as pd
import numpy as np


In [33]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

C:\Users\AX-St\Anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\AX-St\Anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [39]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_csv(filename,sep='|')
    
    #convert to daytime
    df.dropOff_datetime = pd.to_datetime(df.dropOff_datetime)
    df.pickup_datetime = pd.to_datetime(df.pickup_datetime)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [40]:
#df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_????-??.parquet')
df = read_data('../../data/fhv_tripdata_2021-02.csv')

In [41]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [42]:
#QUESTION 1

In [43]:
np.mean(y_pred)

16.191691679979066

In [21]:
#QUESTION 2

In [49]:
year = 2021
month = 2
taxi_type = 'fhv'
run_id =None

input_file = f'../../data/{taxi_type}_tripdata_{year:04d}-{month:02d}.csv'
output_file = f'output/{taxi_type}/{year:04d}-{month:02d}.parquet'


In [53]:
def apply_model(input_file, run_id, output_file,year,month):

    df = read_data(input_file)
    dicts = df[categorical].to_dict(orient='records')
    
    with open('model.bin', 'rb') as f_in:
        dv, lr = pickle.load(f_in)
    
    X_val = dv.transform(dicts)
    y_pred = lr.predict(X_val)
    
    df_result = pd.DataFrame()
    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    df_result['pickup_datetime'] = df['pickup_datetime']
    df_result['PULocationID'] = df['PUlocationID']
    df_result['DOLocationID'] = df['DOlocationID']
    df_result['actual_duration'] = df['duration']
    df_result['predicted_duration'] = y_pred
    df_result['diff'] = df_result['actual_duration'] - df_result['predicted_duration']
    df_result['model_version'] = run_id
    
    df_result.to_parquet(output_file,
    engine='pyarrow',
    compression=None,
    index=False)

In [57]:
apply_model(input_file, run_id, output_file,year,month)

C:\Users\AX-St\Anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\AX-St\Anaconda3\envs\exp-tracking-env\lib\site-packages\sklearn\base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.0.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


  Using cached pyarrow-8.0.0-cp39-cp39-win_amd64.whl (17.9 MB)
